In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
# spark transformation basically has 2 methods Dataframe APi or using spark sql but we wont do spark sql since we know normal sql very well, we will try to understand how to do transformation and important transformations in dataframe api


my_data = [(1,1),
        (2,1),
        (3,1),
        (4,2),
        (5,1),
        (6,2),
        (7,2)]

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import * # for future when needed 

my_schema = ['id', 'num']

In [0]:
df = spark.createDataFrame(data=my_data, schema=my_schema)

In [0]:
df.select("num").show()

+---+
|num|
+---+
|  1|
|  1|
|  1|
|  2|
|  1|
|  2|
|  2|
+---+



In [0]:
# Transformation Starts From below 

In [0]:

df = spark.read.format("csv")\
                .option("header","true")\
                .option("inferschema","true")\
                .option("mode","PERMISSIVE")\
                .load("/FileStore/tables/to_transform_csv.txt")
df.printSchema()
df.show(5)

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT

In [0]:
df.columns

Out[38]: ['EMPLOYEE_ID',
 'FIRST_NAME',
 'LAST_NAME',
 'EMAIL',
 'PHONE_NUMBER',
 'HIRE_DATE',
 'JOB_ID',
 'SALARY',
 'MANAGER_ID',
 'DEPARTMENT_ID']

In [0]:
# How you can typecast or change datatypes of your coloumn

df = df.withColumn("HIRE_DATE", to_date(col("HIRE_DATE"), "dd-MMM-yy"))  #MMM significes that month is in abbrevated name JUN like that
df = df.withColumn("MANAGER_ID", col("MANAGER_ID").cast(IntegerType()))

df.printSchema()
df.show()

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: date (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: integer (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)

+-----------+----------+---------+--------+------------+----------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER| HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+----------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|2007-06-21|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Gra

In [0]:
# How to drop coloumn 

df = df.drop("COMMISSION_PCT") #if multiple use ,
df.show(2)

+-----------+----------+---------+--------+------------+----------+--------+------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER| HIRE_DATE|  JOB_ID|SALARY|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+----------+--------+------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|2007-06-21|SH_CLERK|  2600|       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|2008-01-13|SH_CLERK|  2600|       124|           50|
+-----------+----------+---------+--------+------------+----------+--------+------+----------+-------------+
only showing top 2 rows



In [0]:
# Selecting columns

df.select("EMPLOYEE_ID","FIRST_NAME").show(2) 

# if want to do operation
# Either use withColumn and create a new column or update existing which is recmommened or temporarily use col and add subtract

# For temporary purpose 
df.select(col("EMPLOYEE_ID")+2).show(2)

# using expression in select what this does is convert string type things in select as a job  expression is used for complex transformation which you can do directly with sql
# any select always expects column only Expr will convert the rest sql into logic

df.selectExpr("EMPLOYEE_ID + 2 as EMP").show(2)  # you can write query like logic with Expr



+-----------+----------+
|EMPLOYEE_ID|FIRST_NAME|
+-----------+----------+
|        198|    Donald|
|        199|   Douglas|
+-----------+----------+
only showing top 2 rows

+-----------------+
|(EMPLOYEE_ID + 2)|
+-----------------+
|              200|
|              201|
+-----------------+
only showing top 2 rows

+---+
|EMP|
+---+
|200|
|201|
+---+
only showing top 2 rows



In [0]:
# How alias work
df.select(col("EMPLOYEE_ID").alias("EMP")).show(1)


# Filter how to use filter or where both used for same purpose, these also like select will expect coloumn and the logic
df.filter((col("SALARY")>13000) & (col("MANAGER_ID")==100)).show()
df.where((col("SALARY")>13000) & (col("MANAGER_ID")==100)).show()
# when using $ for and , | for or, you have to enclose your columns in brackets like above 


#what is literal that is lit
# Used when we want to create a coloumn with constant value throughtout the coloumn 

df.select("*", lit("lol").alias("new_col")).show(3)

+---+
|EMP|
+---+
|198|
+---+
only showing top 1 row

+-----------+----------+---------+--------+------------+----------+------+------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER| HIRE_DATE|JOB_ID|SALARY|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+----------+------+------+----------+-------------+
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568|2005-09-21| AD_VP| 17000|       100|           90|
|        102|       Lex|  De Haan| LDEHAAN|515.123.4569|2001-01-13| AD_VP| 17000|       100|           90|
+-----------+----------+---------+--------+------------+----------+------+------+----------+-------------+

+-----------+----------+---------+--------+------------+----------+------+------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER| HIRE_DATE|JOB_ID|SALARY|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+----------+------+------+--------

In [0]:
# withColumn 
# This is used when we want to add a newcoloumn or make changes in existing coloumn like type cast etc
# withColoumnRenamed is used to rename column

df.withColumn("new_emp_is", col("EMPLOYEE_ID")+2)\
    .select("EMPLOYEE_ID","new_emp_is").show(2)
# used withcolumn with select can do complex stuff like this also can use if else and case statements will be shown ahead in future slides

+-----------+----------+
|EMPLOYEE_ID|new_emp_is|
+-----------+----------+
|        198|       200|
|        199|       201|
+-----------+----------+
only showing top 2 rows



In [0]:
# to check for null you can use isNull()

df.filter(col("MANAGER_ID").isNull()).show()

# to replace you can do fillna

df.fillna({"MANAGER_ID":-1}) # if multiple column add comma in between key value pairs key is column name value is what you want to set it with 

# If you want to use dynamic defaults like mean or something you can do it 2 ways 

# Calculate the mean value
mean_value = df.select(mean(col("SALARY"))).collect()[0][0]

# Replace nulls with the mean using withColumn
df.withColumn(
    "SALARY",
    when(col("SALARY").isNull(), mean_value).otherwise(col("SALARY"))
)
#use withcolumn for complex process or also instead of null if you want to replace 
# or 

df.fillna({"SALARY": mean_value})

# Complex operation looks like

df_replaced = df.replace({"pending": "in_progress", "done": "completed"}, subset=["status"])
#subset is in which column to replace

# if want to replace substitute null and everything you can do it together with withColumn whenotherwise

df_transformed = df.withColumn(
    "status",
    when(col("status").isNull(), "Unknown")            # Replace null with "Unknown"
    .when(col("status") == "pending", "in_progress")  # Replace "pending" → "in_progress"
    .otherwise(col("status"))                         # Keep everything else the same
)


+-----------+----------+---------+-----+------------+----------+-------+------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|EMAIL|PHONE_NUMBER| HIRE_DATE| JOB_ID|SALARY|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+-----+------------+----------+-------+------+----------+-------------+
|        100|    Steven|     King|SKING|515.123.4567|2003-06-17|AD_PRES| 24000|      null|           90|
+-----------+----------+---------+-----+------------+----------+-------+------+----------+-------------+

Out[65]: DataFrame[EMPLOYEE_ID: int, FIRST_NAME: string, LAST_NAME: string, EMAIL: string, PHONE_NUMBER: string, HIRE_DATE: date, JOB_ID: string, SALARY: int, MANAGER_ID: int, DEPARTMENT_ID: int]